# Having fun with py-starbound

## Imports:

In [ ]:
import glob
import starbound as sb

import pandas as pd
import seaborn as sns

## Setting the file source

In [ ]:
import configparser

config: configparser.ConfigParser = configparser.ConfigParser()
config.read("config.ini")

starbound_src_folder: str = config.get('src', 'folder')
player_folder: str = f"{starbound_src_folder}/storage/player"
universe_folder: str = f"{starbound_src_folder}/storage/universe"

## Working with character files:

We first load the characters into an *global* array to not need to redo the file operations every time.

In [ ]:
players: list[sb.VersionedJSON] = []
player_files = list(glob.iglob(f"{player_folder}/*.player", recursive=True))

for file in player_files:
	with open(file, 'r+b') as fh:
		player: sb.VersionedJSON = sb.read_sbvj01(fh)
		players.append(player)

### Now a very simple operation to print a specific string to get the characters:

In [ ]:
from utils import playtime_to_string

for player in players:
	name: str = player.data['identity']['name']
	gender: str = player.data['identity']['gender']
	species: str = player.data['identity']['species']
	playtime = playtime_to_string(player.data['log']['playTime'])
	print(f'You got a {gender} {species.title()} with the name: "{name}" that you played for: {playtime}')

### Dumping the JSON Data into a json file:

In [ ]:
# import json

# for player in players:
# 	name: str = player.data['identity']['name']
# 	with open(f'{name}.json', 'w', encoding='utf-8') as f:
# 		json.dump(player.data, f, ensure_ascii=False, indent=4)

## Working with world files:

We first load the worlds into an *global* array to not need to redo the file operations every time.

In [ ]:
import mmap

worlds: list[sb.World] = []

world_files = list(glob.iglob(f"{universe_folder}/*_*_*.world", recursive=True))
for file in world_files:
	with open(file, 'r+b') as fh:
		mm: mmap = mmap.mmap(fh.fileno(), 0, access=mmap.ACCESS_READ)
		world: sb.World = sb.World(mm)
		# world.read_metadata()
		worlds.append(world)

### Printing all the important world details

In [ ]:
from utils import strip_colors

for world in worlds:
	print(f'World Name: {strip_colors(world.info.name)}')
	print(f'World size: {world.width}×{world.height}')
	print(f'World Description: {world.info.description}')
	print(f'--------------------------------------------------------------------------------')
	# x, y = world.metadata['playerStart']
	# print(f'Player spawns at ({x}, {y})')
	# Some funny stuff with getting entities...
	# try:
	# 	rx, ry = x // 32, y // 32
	# 	# print(f'Spawn Region is: ({rx}, {ry})')
	# 	entities = world.get_entities(int(rx), int(ry))
	# 	for entity in entities:
	# 		if entity.name == 'ObjectEntity':
	# 			items = entity.data.get("items")
	# 			for item in items:
	# 				if item == None:
	# 					break
	# 				print(item)
	# # Just ignoring all Exceptions...
	# except Exception:
	# 	pass

### Getting the Amount of dungeons

In [ ]:
known_dungeons: dict[str, int] = {}

for world in worlds:
	dungeons: list[str] = world.info.dungeons
	for dungeon in dungeons:
		if dungeon in known_dungeons:
			known_dungeons[dungeon] = known_dungeons[dungeon] + 1
		else:
			known_dungeons[dungeon] = 1

dungeons = pd.Series(data=known_dungeons).explode().reset_index(name="times").rename(columns={"index": "dungeons"}).sort_values("times")

sns.catplot(
	data=dungeons,
	kind="bar",
	x="times",
	y="dungeons",
	height=10
)

### Getting the amount of biomes

In [ ]:

known_biomes: dict[str, int]  = {}

for world in worlds:
	biomes: list[str] = world.info.biomes
	for biome in biomes:
		if "underground" in biome or "asteroids" in biome or "atmosphere" in biome or "core" in biome or "oceanfloor" in biome or "rust" in biome:
			continue
		if biome in known_biomes:
			known_biomes[biome] = known_biomes[biome] + 1
		else:
			known_biomes[biome] = 1

biomes = pd.Series(data=known_biomes).explode().reset_index(name="times").rename(columns={"index": "biomes"}).sort_values("times")

sns.catplot(
	data=biomes,
	kind="bar",
	x="times",
	y="biomes",
	height=10
)